In [228]:
import numpy as np
import pandas as pd
import math
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

x,y = make_classification(n_samples=150, n_features=4, n_classes=2, n_redundant=0, n_informative=4, random_state=5)
x = pd.DataFrame(x , columns=['1', '2', '3', '4'])
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.3,random_state=4)

def dataset(a):
    f = 0
    for i in a.index:
        if(i==a.index[0]):
            a.loc[i,'buckets'] = a.loc[i, 'weights']
        else:
            f+=1
            j = a.index[f-1]
            a.loc[i,'buckets'] = a.loc[j, 'buckets'] + a.loc[i, 'weights']

    dfn = pd.DataFrame(data = np.zeros((a.shape[0], a.shape[1])), columns=a.columns, index=a.index)
    dum = pd.concat([pd.Series(0),a.buckets])
    b = list(zip(dum, a.buckets))

    #n = np.random.rand()
    #v = [i[1] for i in b if (i[0]<n) & (i[1]>n)]
    #pos = a.index[a.buckets==v[0]]
    #dfn.loc[pos] = a.loc[pos].copy()
    
    for j in range(len(dfn)):
        n = np.random.rand()
        v = [i[1] for i in b if (i[0]<n) & (i[1]>n)]
        pos = a.index[a.buckets==v[0]]
        dfn.loc[pos] = a.loc[pos].copy()

    dfn = dfn.drop('buckets', axis='columns')
    return dfn
    
    #dfn.drop('buckets', axis='columns', inplace=True)
    #return dfn



def adaboost(a,b):
    c = a.copy()
    c.drop('weights', axis='columns', inplace=True)
    #print(c.columns)
    dtr = DecisionTreeClassifier(criterion='gini', max_depth=1, random_state=4).fit(c,b)  # stump
    prediction = dtr.predict(c)
    te = np.sum(a.weights[prediction!=b])
    
    while(len([prediction!=b])>2):
    #if (te==0):
        #return 1
    #else:
        #say = 0.5*(math.log((1-te)/te))
        
        say = 0.5*(math.log((1-te)/te))
        mis = a.index[prediction!=b]
        for index in mis:
            v = a.loc[index, 'weights']
            a.loc[index, 'weights'] = v*math.exp(say)
    
        notmis = a.index[prediction==b]
        for index in notmis:
            v = a.loc[index, 'weights']
            a.loc[index, 'weights'] = v*math.exp(-say)
        
        a['weights'] = a.weights / np.sum(a.weights) # normalizing
        #r = dtr.predict(t1)
        #print(r, say, end=' ')
        print('-')
    #print(res, say, end=' ')
        new = dataset(a)
        adaboost(new, ytrain)
        
        
    
    return dtr 
    
    


xtrain['weights'] = 1/xtrain.shape[0]
res = adaboost(xtrain, ytrain)
print(res.predict(xtest))
p = res.predict(xtest)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(ytest,p)
print('Accuracy - {}'.format(accuracy))

[1 1 1 0 1 1 0 1 1 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 1 1 0
 1 0 1 1 0 1 1 1]
Accuracy - 0.7111111111111111


In [151]:
xtrain.columns

Index(['1', '2', '3', '4', 'weights', 'buckets'], dtype='object')

In [153]:
Xtrain = xtrain.drop(['weights', 'buckets'], axis='columns')

In [233]:
from sklearn.ensemble import AdaBoostClassifier
abc = AdaBoostClassifier(n_estimators=100, random_state=4).fit(Xtrain, ytrain)
res = abc.predict(xtest)
print(res)
a1 = accuracy_score(ytest, res)
print('Accuracy - {}'.format(a1))

[1 1 1 0 0 1 0 0 1 0 1 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 1 0
 1 1 1 1 0 1 0 1]
Accuracy - 0.7111111111111111
